In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AMAZONINSTRUMENTS").getOrCreate()

In [3]:

from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [10]:
cleaned_df=df.dropna()
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [5]:
parent_df=cleaned_df.select(["review_id", "customer_id", "product_id","product_parent","review_date"])
parent_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RMDCHWD0Y5OZ9|   45610553|B00HH62VB6|     618218723| 2015-08-31|
| RZSL0BALIYUNU|   14640079|B003LRN53I|     986692292| 2015-08-31|
| RIZR67JKUDBI0|    6111003|B0006VMBHI|     603261968| 2015-08-31|
|R27HL570VNL85F|    1546619|B002B55TRG|     575084461| 2015-08-31|
|R34EBU9QDWJ1GD|   12222213|B00N1YPXW2|     165236328| 2015-08-31|
|R1WCUI4Z1SIQEO|   46018513|B001N4GRGS|     134151483| 2015-08-31|
| RL5LNO26GAVJ1|   10225065|B009PJRMHQ|     694166585| 2015-08-31|
|R3GYQ5W8JHP8SB|    6356995|B00NKBDAZS|     446431775| 2015-08-31|
|R30SHYQXGG5EYC|   35297198|B006MIU7U2|     125871705| 2015-08-31|
|R14YLXA56NP51I|   32139520|B000FIBD0I|     771888534| 2015-08-31|
|R1ZH0HSH38IOTZ|   36060782|B0002E52GG|      68535945| 2015-08-31|
|R3H53KLLC210XI|    5301309|B00RZIH52G|     725541773| 2015-08

In [6]:
product_df=cleaned_df.select(["product_id", "product_title"])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00HH62VB6|AGPtek® 10 Isolat...|
|B003LRN53I|Sennheiser HD203 ...|
|B0006VMBHI|AudioQuest LP rec...|
|B002B55TRG|Hohner Inc. 560BX...|
|B00N1YPXW2|Blue Yeti USB Mic...|
|B001N4GRGS|Middle Atlantic P...|
|B009PJRMHQ|Kmise 1pc Pickgua...|
|B00NKBDAZS|Kealoha Concert U...|
|B006MIU7U2|Halco 80000 - MR1...|
|B000FIBD0I|Gator GPTBLACK Pl...|
|B0002E52GG|Hetman 1 - Light ...|
|B00RZIH52G|Dragonpad pop fil...|
|B001792BAU|DharmaObjects Rel...|
|B009GSKW1Y|Musiclily SSS Pla...|
|B0002F4TKA|Vic Firth America...|
|B00K17YFBW|Guitar Stand for ...|
|B00EQ24HJS|Generic 3PLY Faux...|
|B00IBOYTUE|Audio 2000 6525 3...|
|B00FBRUSAE|Sawtooth ST-AMP-1...|
|B0113D2QUO|Upado Unlimited G...|
+----------+--------------------+
only showing top 20 rows



In [9]:
customer_df=cleaned_df.groupby("customer_id").count()
customer_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    9163460|    2|
|   50809283|    1|
|   13005724|    1|
|   51506042|    1|
|   48752158|   22|
|   12318800|    2|
|   26246534|    2|
|   47577568|    1|
|   15963400|   10|
|     134314|    1|
|   32457519|    1|
|   23389022|    9|
|   32604233|    1|
|    3430931|    8|
|   22387890|    2|
|     847650|    1|
|   14633838|    3|
|   11886080|    1|
|    2664669|    1|
|   23389273|    1|
+-----------+-----+
only showing top 20 rows



In [8]:
vinetable_df=cleaned_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vinetable_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|
| RZSL0BALIYUNU|          5|            0|          0|   N|
| RIZR67JKUDBI0|          3|            0|          1|   N|
|R27HL570VNL85F|          5|            0|          0|   N|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|
|R30SHYQXGG5EYC|          5|            0|          0|   N|
|R14YLXA56NP51I|          5|            1|          1|   N|
|R1ZH0HSH38IOTZ|          5|            0|          0|   N|
|R3H53KLLC210XI|          4|            0|          0|   N|
|R3OOR877NGA8JK|          3|            0|          0|   N|
|R1BY7WKOZ3KMH0|          2|            

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
parent_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vinetable_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)